In [1]:
import pandas as pd
from pathlib import Path
import json

In [2]:
indir = Path("/home/nico/Software/nf-core/tfactivity/testing/work/c1/ef45dc7e9a1f8a857ffee79ff738e3")
outdir = Path("src/assets")

In [3]:
df_design = pd.read_csv(indir / "counts_design/samplesheet_rnaseq.csv")

# Group by condition, get sample list
sample_condition = df_design.groupby("condition").apply(lambda x: x["sample"].tolist()).to_dict()
json.dump(sample_condition, open(outdir / "sample_condition.json", "w"), indent=4)

/tmp/ipykernel_229020/926323149.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_condition = df_design.groupby("condition").apply(lambda x: x["sample"].tolist()).to_dict()


In [4]:
affinity_ratio_dir = indir / "affinity_ratio"
assert affinity_ratio_dir.exists()

affinity_sum_dir = indir / "affinity_sum"
assert affinity_sum_dir.exists()

deseq2_differential_dir = indir / "deseq2_differential"
assert deseq2_differential_dir.exists()

normalized_counts_dir = indir / "normalized"
assert normalized_counts_dir.exists()

raw_counts_dir = indir / "raw_counts"
assert raw_counts_dir.exists()

regression_coefficients_dir = indir / "regression_coefficients"
assert regression_coefficients_dir.exists()

tf_ranking_dir = indir / "tf_rankings"
assert tf_ranking_dir.exists()

tg_ranking_dir = indir / "tg_rankings"
assert tg_ranking_dir.exists()

tpm_dir = indir / "tpms"
assert tpm_dir.exists()

In [5]:
ranking = {}
tfs = {}


for file in tf_ranking_dir.glob("*.tf_ranking.tsv"):
    assay = file.stem.split(".")[0]
    
    df_tf = pd.read_csv(file, sep="\t", index_col=0)
    ranking[assay] = df_tf["dcg"].to_dict()

    df_tg = pd.read_csv(tg_ranking_dir / f"{assay}.tg_ranking.tsv", sep="\t", index_col=0)

    for tf in df_tf.index:
        if tf not in tfs:
            tfs[tf] = {
                "target_genes": {},
                "differential_expression": {}
            }
        tfs[tf]["target_genes"][assay] = df_tg[tf].to_dict()

for file in deseq2_differential_dir.glob("*.deseq2.results.tsv"):
    pair_string = file.stem.split(".")[0]
    
    df_deseq2 = pd.read_csv(file, sep="\t", index_col=0)

    for tf in tfs:
        tf_row = df_deseq2.loc[tf]
        tfs[tf]["differential_expression"][pair_string] = {
            "baseMean": tf_row["baseMean"],
            "log2FoldChange": tf_row["log2FoldChange"],
            "lfcSE": tf_row["lfcSE"],
            "pvalue": tf_row["pvalue"],
        }

json.dump(ranking, open(outdir / "ranking.json", "w"), indent=4)

tf_dir = outdir / "transcription_factors"
tf_dir.mkdir(parents=True, exist_ok=True)

df_tpm = pd.read_csv(tpm_dir / "counts.tpm.tsv", sep="\t", index_col=0).T
df_counts = pd.read_csv(raw_counts_dir / "counts.counts_filtered.tsv", sep="\t", index_col=0).T

for tf in tfs:
    tfs[tf]["tpm"] = df_tpm[tf].to_dict()
    tfs[tf]["counts"] = df_counts[tf].to_dict()

    json.dump(tfs[tf], open(tf_dir / f"{tf}.json", "w"), indent=4)